# Advent of Code Day 7

On Day 7, we need to determine the base tower in a graph-style configuration (part one) and then find the imbalanced tower within the graph (part two).  

In [ ]:
from utils import read_input

In [ ]:
def load_graph(filename):

    import re
    
    bottom_pattern = re.compile('^.*?\s+')
    bottom_weight = re.compile('[0-9]+')
    top_pattern = re.compile('(?<=->)(.+)$')
    
    dependencies = read_input(filename)
    
    weights = {}
    graph = {}        
    
    for dep in dependencies:    
        
        tower = bottom_pattern.findall(dep)[0].strip()
        weight = bottom_weight.findall(dep)[0]
        top = top_pattern.findall(dep)
        
        graph[tower] = set([t.strip() for t in top[0].split(',')]) if top else set([])
        weights[tower] = int(weight)      
        
    return Graph(graph, weights)

In [ ]:
class Graph(object):
    
    def __init__(self, adjacency_list, weights):
        root_name = self._get_root(adjacency_list)
        
        self.root = Node(root_name, weights[root_name])
        
        parent = self.root       
            
        self._attach_child_nodes(self.root, adjacency_list, weights)
    
    def _attach_child_nodes(self, parent_node, adjacency_list, weights):
        children = adjacency_list[parent_node.my_name()]
        
        for child in children:
            child_node = Node(child, weights[child])
            self._attach_child_nodes(child_node, adjacency_list, weights)            
            parent_node.add_child(child_node)        
      
    def _get_root(self, adjacency_list):
        # Combine all the referents together into one set
        referents = reduce(lambda f, s: f.union(s), [v for k,v in adjacency_list.items()])
    
        # The bottom tower is the one that is in the graph but is not referenced (a referent)
        return list(set(adjacency_list.keys()).difference(referents))[0]
    

class Node(object):
    
    def __init__(self, name, weight):
        self._name = name
        self._weight = weight
        self.children = set([])
    
    def add_child(self, child_node):
        self.children.add(child_node)    
        
    def my_name(self):
        return self._name
        
    def my_weight(self):
        return self._weight
        
    def weight(self):
        child_weights = [c.weight() for c in self.children]
        
        return self._weight + sum(child_weights)
   
        

In [ ]:
def whose_on_bottom(graph):    
    
    # Combine all the referents together into one set
    referents = reduce(lambda f, s: f.union(s), [v for k,v in graph.items()])
    
    # The bottom tower is the one that is in the graph but is not referenced (a referent)
    return set(graph.keys()).difference(referents)    

In [ ]:
def visit(graph):
    
    def visit(node, level = 0):
    
        prefix = '\t' * level
    
        weights = [c.weight() for c in node.children]
        if weights != [] and len(set(weights)) > 1:
            print '{}{} ({}) [{}]'.format(prefix, node.my_name(), node.my_weight(),  weights)
        
        for child in node.children:
            visit(child, level + 1)
            
    visit(graph.root)


In [ ]:
g = load_graph('Input/day7.txt')

visit(g)